# 智能垃圾分类系统 - 完整实验流程
# Intelligent Waste Classification System - Complete Workflow

**基于迁移学习和MobileNetV2的智能垃圾分类系统**

本Notebook包含完整的实验流程：
- 📦 环境配置
- 📥 数据集下载
- 🔧 数据预处理
- 🧠 模型训练（MobileNetV2、ResNet50、VGG16、DenseNet121、EfficientNetB0）
- 📊 结果评估与可视化
- 🔬 消融实验（CBAM注意力机制 + Focal Loss）

---

## 📋 目录
1. [环境设置](#1-环境设置)
2. [项目克隆与依赖安装](#2-项目克隆与依赖安装)
3. [Kaggle数据集下载](#3-kaggle数据集下载)
4. [数据探索](#4-数据探索)
5. [模型训练](#5-模型训练)
6. [结果评估](#6-结果评估)
7. [消融实验（可选）](#7-消融实验可选)
8. [下载结果](#8-下载结果)

---
## 1. 环境设置

检查GPU环境，设置运行环境

In [ ]:
# 检查GPU
!nvidia-smi

import tensorflow as tf
print("TensorFlow版本:", tf.__version__)
print("GPU可用:", tf.config.list_physical_devices('GPU'))

# 设置GPU内存增长
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"检测到 {len(gpus)} 个GPU，已启用内存增长")
    except RuntimeError as e:
        print(f"GPU配置错误: {e}")

---
## 2. 项目克隆与依赖安装

从GitHub克隆项目代码

In [ ]:
# 克隆项目（如果需要）
# 方式1：从GitHub克隆
# !git clone https://github.com/FKNLEDGE/dection_grb-1.git
# %cd dection_grb-1

# 方式2：如果已上传文件到Colab，跳过克隆步骤
# 确保当前目录包含项目文件

import os
# 如果在项目目录中，显示文件列表
!ls -la

In [ ]:
# 安装依赖包
!pip install -q tensorflow>=2.10.0
!pip install -q numpy pandas scikit-learn
!pip install -q matplotlib seaborn
!pip install -q Pillow opencv-python
!pip install -q tqdm
!pip install -q kaggle

print("✅ 依赖安装完成！")

---
## 3. Kaggle数据集下载

### 3.1 上传Kaggle API Token

**获取Kaggle API Token步骤：**
1. 访问 [Kaggle](https://www.kaggle.com)
2. 点击右上角头像 → Settings
3. 找到 "API" 部分，点击 "Create New Token"
4. 下载 `kaggle.json` 文件
5. 在下方单元格上传该文件

In [ ]:
# 上传kaggle.json文件
from google.colab import files

print("请上传您的 kaggle.json 文件：")
uploaded = files.upload()

# 配置Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\n✅ Kaggle API配置完成！")

### 3.2 下载垃圾分类数据集

In [ ]:
# 下载数据集
!kaggle datasets download -d mostafaabla/garbage-classification

# 解压数据集
!unzip -q garbage-classification.zip -d ./data

# 检查数据集结构
import os

# 处理可能的目录结构差异
if os.path.exists('./data/Garbage classification'):
    !mv './data/Garbage classification' ./data/garbage_classification
elif os.path.exists('./data/garbage_classification'):
    print("数据集目录已存在")
else:
    # 尝试查找数据集位置
    !find ./data -type d -maxdepth 2

# 显示数据集类别
data_dir = './data/garbage_classification'
if os.path.exists(data_dir):
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    print(f"\n✅ 数据集下载完成！共 {len(classes)} 个类别：")
    for i, cls in enumerate(classes, 1):
        count = len(os.listdir(os.path.join(data_dir, cls)))
        print(f"  {i}. {cls}: {count} 张图片")
else:
    print("❌ 数据集目录未找到，请检查解压路径")

---
## 4. 数据探索

可视化数据集样本

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random

data_dir = './data/garbage_classification'
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

# 显示每个类别的一张示例图片
fig, axes = plt.subplots(3, 4, figsize=(15, 12))
axes = axes.flatten()

for idx, cls in enumerate(classes):
    cls_dir = os.path.join(data_dir, cls)
    images = [f for f in os.listdir(cls_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if images:
        sample_img = random.choice(images)
        img_path = os.path.join(cls_dir, sample_img)
        img = Image.open(img_path)
        
        axes[idx].imshow(img)
        axes[idx].set_title(f"{cls}\n({len(images)} images)", fontsize=10)
        axes[idx].axis('off')

plt.tight_layout()
plt.savefig('dataset_samples.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ 数据集样本可视化完成！")

---
## 5. 模型训练

### 5.1 快速测试（3个epoch）

先用少量epoch快速测试代码是否正常运行

In [ ]:
# 快速测试模式
from main import run_quick_test

print("=" * 70)
print("开始快速测试（3 epochs，仅MobileNetV2）")
print("=" * 70)

try:
    results, models, histories = run_quick_test(
        data_dir='./data/garbage_classification',
        epochs=3,
        models=['MobileNetV2']
    )
    print("\n✅ 快速测试成功！代码运行正常。")
except Exception as e:
    print(f"\n❌ 快速测试失败: {e}")
    import traceback
    traceback.print_exc()

### 5.2 完整训练（可选）

运行完整的多模型对比实验（30个epoch）

**⚠️ 注意：完整训练需要较长时间（约1-3小时），建议在有GPU的环境下运行**

In [ ]:
# 完整训练 - 取消下面的注释来运行
# from main import run_all_experiments

# print("=" * 70)
# print("开始完整训练（30 epochs，5个模型）")
# print("预计时间：1-3小时（取决于GPU性能）")
# print("=" * 70)

# try:
#     results, models, histories = run_all_experiments(
#         data_dir='./data/garbage_classification',
#         epochs=30
#     )
#     print("\n✅ 完整训练成功！")
# except Exception as e:
#     print(f"\n❌ 训练失败: {e}")
#     import traceback
#     traceback.print_exc()

print("提示：如需运行完整训练，请取消上方单元格的注释")

### 5.3 单模型训练

训练单个指定的模型

In [ ]:
# 单模型训练示例
from main import run_all_experiments

# 选择要训练的模型（可选：'MobileNetV2', 'MobileNetV2_CBAM', 'VGG16', 'DenseNet121', 'EfficientNetB0'）
selected_model = 'MobileNetV2_CBAM'  # 带CBAM注意力机制的MobileNetV2

print(f"\n训练模型: {selected_model}")
print("=" * 70)

try:
    results, models, histories = run_all_experiments(
        data_dir='./data/garbage_classification',
        epochs=20,  # 可调整epoch数
        models_to_compare=[selected_model]
    )
    print(f"\n✅ {selected_model} 训练完成！")
except Exception as e:
    print(f"\n❌ 训练失败: {e}")
    import traceback
    traceback.print_exc()

---
## 6. 结果评估

查看训练结果和可视化图表

In [ ]:
# 显示训练结果
import pandas as pd
from IPython.display import display, Image as IPImage

# 读取模型对比结果
comparison_file = './outputs/model_comparison.csv'
if os.path.exists(comparison_file):
    df = pd.read_csv(comparison_file)
    print("\n📊 模型对比结果：")
    print("=" * 70)
    display(df)
else:
    print("⚠️ 未找到对比结果文件，请先运行训练")

# 显示最佳模型
if os.path.exists(comparison_file):
    df = pd.read_csv(comparison_file)
    best_acc = df.loc[df['accuracy'].idxmax()]
    best_speed = df.loc[df['inference_time_ms'].idxmin()]
    
    print("\n🏆 最佳准确率模型：")
    print(f"  模型: {best_acc['model_name']}")
    print(f"  准确率: {best_acc['accuracy']:.4f}")
    print(f"  F1分数: {best_acc['f1_score']:.4f}")
    
    print("\n⚡ 最快推理速度模型：")
    print(f"  模型: {best_speed['model_name']}")
    print(f"  推理时间: {best_speed['inference_time_ms']:.2f} ms")
    print(f"  FPS: {best_speed['fps']:.1f}")

In [ ]:
# 显示混淆矩阵和训练曲线
import matplotlib.pyplot as plt
from IPython.display import Image as IPImage, display

# 查找所有输出图片
output_dirs = [d for d in os.listdir('./outputs') if os.path.isdir(os.path.join('./outputs', d))]

for model_dir in output_dirs:
    print(f"\n{'='*70}")
    print(f"模型: {model_dir}")
    print(f"{'='*70}")
    
    model_path = os.path.join('./outputs', model_dir)
    
    # 显示混淆矩阵
    cm_path = os.path.join(model_path, 'confusion_matrix.png')
    if os.path.exists(cm_path):
        print("\n📊 混淆矩阵：")
        display(IPImage(filename=cm_path))
    
    # 显示训练曲线
    tc_path = os.path.join(model_path, 'training_curves.png')
    if os.path.exists(tc_path):
        print("\n📈 训练曲线：")
        display(IPImage(filename=tc_path))

---
## 7. 消融实验（可选）

验证CBAM注意力机制和Focal Loss的贡献

**⚠️ 注意：消融实验需要训练4个模型，耗时较长**

In [ ]:
# 消融实验 - 取消下面的注释来运行
# from main import run_ablation_study

# print("=" * 70)
# print("开始消融实验")
# print("配置：")
# print("  1. Baseline (无CBAM，无Focal Loss)")
# print("  2. + CBAM (仅CBAM)")
# print("  3. + Focal Loss (仅Focal Loss)")
# print("  4. + CBAM + Focal Loss (完整提出方法)")
# print("=" * 70)

# try:
#     ablation_results = run_ablation_study(
#         data_dir='./data/garbage_classification',
#         epochs=20
#     )
#     print("\n✅ 消融实验完成！")
# except Exception as e:
#     print(f"\n❌ 消融实验失败: {e}")
#     import traceback
#     traceback.print_exc()

print("提示：如需运行消融实验，请取消上方单元格的注释")

---
## 8. 下载结果

打包并下载所有实验结果

In [ ]:
# 打包结果文件
!zip -r experiment_results.zip ./outputs ./saved_models ./logs

print("\n✅ 结果已打包为 experiment_results.zip")
print("\n包含内容：")
print("  📁 outputs/ - 可视化图表和评估结果")
print("  📁 saved_models/ - 训练好的模型文件")
print("  📁 logs/ - TensorBoard日志")

# 下载结果
from google.colab import files
files.download('experiment_results.zip')

---
## 🎉 实验完成！

### 📝 结果说明

本实验对比了多种深度学习模型在垃圾分类任务上的表现：

**标准模型：**
- **MobileNetV2**: 轻量级模型，适合移动端部署
- **VGG16**: 经典深度网络
- **DenseNet121**: 密集连接网络
- **EfficientNetB0**: 高效网络架构

**改进模型：**
- **MobileNetV2_CBAM**: 添加CBAM注意力机制的MobileNetV2

### 📊 关键指标

- **准确率 (Accuracy)**: 分类正确的样本比例
- **F1分数 (F1-Score)**: 精确率和召回率的调和平均
- **推理时间 (Inference Time)**: 单张图片的处理时间
- **模型大小 (Model Size)**: 模型文件大小

### 🔍 进一步探索

1. **TensorBoard可视化**: 
   ```python
   %load_ext tensorboard
   %tensorboard --logdir logs/
   ```

2. **模型预测示例**:
   ```python
   from tensorflow.keras.models import load_model
   model = load_model('./saved_models/MobileNetV2_final.keras')
   # 进行预测...
   ```

3. **查看详细评估报告**:
   - 混淆矩阵: `outputs/<model_name>/confusion_matrix.png`
   - 训练曲线: `outputs/<model_name>/training_curves.png`
   - 数值结果: `outputs/<model_name>/results.json`

---

### 📚 参考资料

- [MobileNetV2 论文](https://arxiv.org/abs/1801.04381)
- [CBAM 论文](https://arxiv.org/abs/1807.06521)
- [Focal Loss 论文](https://arxiv.org/abs/1708.02002)

---

**如有问题，请查看项目GitHub仓库或提交Issue**

---
## 附录：高级功能

### A. 测试单张图片预测

In [ ]:
# 测试单张图片预测
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# 加载模型
model_path = './saved_models/MobileNetV2_final.keras'  # 修改为你的模型路径

if os.path.exists(model_path):
    model = load_model(model_path, custom_objects={'FocalLoss': lambda **kwargs: None}, compile=False)
    
    # 类别名称
    class_names = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes', 
                   'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']
    
    # 上传测试图片
    uploaded = files.upload()
    
    for fn in uploaded.keys():
        # 加载图片
        img = image.load_img(fn, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        
        # 预测
        predictions = model.predict(img_array)
        predicted_class = class_names[np.argmax(predictions)]
        confidence = np.max(predictions) * 100
        
        # 显示结果
        plt.figure(figsize=(10, 4))
        
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title(f"预测类别: {predicted_class}\n置信度: {confidence:.2f}%")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.barh(class_names, predictions[0])
        plt.xlabel('概率')
        plt.title('各类别预测概率')
        plt.tight_layout()
        plt.show()
else:
    print(f"❌ 模型文件不存在: {model_path}")
    print("请先运行训练代码")

### B. TensorBoard可视化

In [ ]:
# 启动TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/

### C. 模型性能分析

In [ ]:
# 分析模型复杂度和推理速度
import pandas as pd
import matplotlib.pyplot as plt

comparison_file = './outputs/model_comparison.csv'
if os.path.exists(comparison_file):
    df = pd.read_csv(comparison_file)
    
    # 准确率 vs 模型大小
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.scatter(df['model_size_mb'], df['accuracy'], s=100, alpha=0.6)
    for i, model in enumerate(df['model_name']):
        plt.annotate(model, (df['model_size_mb'][i], df['accuracy'][i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.xlabel('模型大小 (MB)')
    plt.ylabel('准确率')
    plt.title('准确率 vs 模型大小')
    plt.grid(alpha=0.3)
    
    # 准确率 vs 推理时间
    plt.subplot(1, 2, 2)
    plt.scatter(df['inference_time_ms'], df['accuracy'], s=100, alpha=0.6, color='orange')
    for i, model in enumerate(df['model_name']):
        plt.annotate(model, (df['inference_time_ms'][i], df['accuracy'][i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    plt.xlabel('推理时间 (ms)')
    plt.ylabel('准确率')
    plt.title('准确率 vs 推理时间')
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('model_tradeoff_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ 性能分析图已保存为 model_tradeoff_analysis.png")
else:
    print("❌ 未找到对比结果文件")